## Guia de Resolucion de Proyecto Integrador

### Objetivo
Poder ofrecer de guia y explicacion al trabajo integrador ofrecido por SoyHenry, desde ahora nombrado como la institucion, en el modulo 4 de Data Science, relacionado con las distintas herramientas de Big Data y auxiliares que permiten una introduccion al trabajo de grandes volumenes de datos mediante un equipo conjunto de trabajo desde un servidor remoto.

### Metodologia
Vamos a seguir paso a paso la resolucion de los ejercicios segun planteados, utilizando el mismo material que se encuentra en la carpeta '/DS-M4-Herramientas_Big_Data', aunqune en el transcurso del trabajo veremos como poder modificarla dentro del entorno Linux de nuestra maquina virtual.

### Pasos previos
Es necesario tener una maquina virtual de al menos 60 GB de memoria, que utilice Linux Ubuntu. En caso de tener la dada por la institucion, debemos incrementar el espacio de la misma siguiendo los pasos del archivo 'Resizing_VM.md'.
Ademas, para emular una conexion a un servidor remoto y para una mejor interaccion con la consola de la VM, vamos a utilizar Putty, cuya instalacion no vamos a explicar en este practico.

### Ejercicio 1: Entorno HDFS

Hadoop y, principalmente hdfs, es una herramienta que permite la accesibilidad y consistencia de la informacion dentro de un conjunto de servidores.

Una ves iniciada la maquina virtual con la nueva memoria de 60 GB, avanzaremos con el paso inicial... Para poder ejecutar los archivos dados, debemos descargarlos dentro de la maquina virtual, ya que la misma no tiene ninguna conexion con el CPU Windows o Mac que estemos utilizando.

Para realizar esto podemos hacerlo de dos formas, utilizando git o mediante WinSCP. Dado que WinSCP es una redundancia ya que habria que usar git de todas formas, dejaremos esta herramienta para una explicacion futura.


In [ ]:
git clone https://github.com/lopezdar222/herramientas_big_data # este codigo descarga la carpeta de trabajo en la VM
ls # atributo de listado, en este caso listar carpetas del directorio actual, verificar que exista 'DS-M4-Herramientas_Big_Data'
cd DS-M4-Herramientas_Big_Data # esto basicamente es el comando para ingresar a la carpeta homonima
sudo docker-compose -f docker-compose-v1.yml up -d 
# el comando docker-compose es para crear multiples contenedores a partir de un archivo yml, por defecto docker-compose.yml.
# agregamos -f para indicar el nombre del archivo a ejecutar y evitar el por defecto, y el -d para poder mantenernos en consola para seguir ejecutando codigo. 

Daremos una breve explicacion de lo ocurrido en este paso por unica vez al realizar un docker-compose para que se pueda entender este misterio... Para ello, ver el archivo docker-compose-v1.yml en Windows o en el mismo GitHub. <br>
Veremos que utilizando el docker-compose version 3, llamamos a describir diversos contenedores, cada uno con su imagen relacionada y otros atributos:<br>
<ul>
    <li>Namenode: este contenedor es el administrador de metadata y particiones del sistema de hadoop. El mismo tiene una imagen ya creada a utilizar y dos puertos de conexion (veremos que se conectan con varios contenedores). Ademas, cuenta con dos volumenes en formato hdfs (ya lo veremos) para poder resguardar la inforamcion en caso de reinicio o desaparicion del mismo dentro de la maquina virtual.</li>
    <li>Datanode: este nodo es quien almacena la informacion particionada que le brinda Namenode.. en este caso las particiones son todas sin redundancia por no tener nodos esclavos (o copias). Se conecta a Namenode mediante el puerto 9870 y en el entorno hdfs al namenode mediante el puerto 9000. Finalmente, tiene el puerto 9864 abrierto para conexiones hacia el mismo.</li>
    <li>Resourcemanager: es el contenedor que optimiza el uso de los datanodes, sirviendo como interactor entre Namenode y Datanode.</li>
    <li>Historyserver: tambien se conecta con Namenode y Datanode pero en este caso con el fin de llevar un historial de las operatorias de MapReduce, se utiliza para eficientizar operaciones.</li>
<br></ul>
Entonces, este docker-compose genera esta red de contenedores interconectados para su posterior utilizacion, en este caso dentro del entorno hadoop.

In [ ]:
sudo docker cp Datasets namenode:/home/ #copia la carpeta entera dentro del contenedor namenode, carpeta home
sudo docker exec -it namenode bash #ingresas al contenedor namenode (exec es para ejecutar un contenedor corriendo, -it para que sea iteractivo, namenode es el nombre del contenedor y bash es para decirle que queres trabajar sobre el bash del programa)
cd home #ingresas a la carpeta donde se copio Datasets
ls #muestra que hay en dicho directorio
hdfs dfs -mkdir -p /data #hdfs dfs te conecta con el entorno de archivos particionados hadoop, -mkdir es para crear un directorio, -p es para crear las carpetas madres necesarias y /data es el nombre del directorio
hdfs dfs -put /home/Datasets/* /data #en este caso estas particionando y generando redundancia (aunque no la haya por ser solo un contenedor de Datanode) a los archivos existentes en Datasets dentro del entorno hadoop
exit #para salir del contenedor y volver a nuestra carpeta dentro de la VM

Con este codigo, obtenemos los archivos en Datasets particionados y replicados segun indica la configuracion del contenedor namenode.

### Ejercicio 2: Hive

Hive es una herramienta que permite la gestion de bases de datos relacionales dentro del entorno de hadoop y archivos particionados.

En este paso aprenderemos a ejecutar un script Hive dentro de un contenedor, cuyo objetivo es crear tablas a partir de nuestros Datasets DENTRO DEL ENTORNO HDFS y trabajar sobre ellas.    

In [ ]:
sudo docker-compose -f docker-compose-v2.yml up -d #ejecutar el docker-compose-v2 que es un poco mas completo que el 1, pero con la misma sintaxis apra que puedas investigar usando ChatGPT
sudo docker cp Paso02.hql hive-server:/opt/Paso02.hql #copias el archivo dado para ejecutar las tablas del entorno hdfs dentro de hive
sudo docker exec -it hive-server bash #como hicimos en el punto anterior, nos metemos dentro del contenedor
hive -f Paso02.hql #dentro del contenedor, con este comando ejecutas el programa Hive y haces que corra el script Paso02.hql
exit #vuelve a la consola normal

### Ejecicio 3: Formatos de Almacenamiento

Ahora vamos a buscar convertir estas bases de dato a archivos con otros formatos de almacenamiento:

In [ ]:
sudo docker cp Paso03.hql hive-server:/opt/Paso03.hql #copiamos el archivo para el paso 3
sudo docker exec -it hive-server bash
ls #verificamos que este el archivo Paso03.hql
hive -f Paso03.hql
hive #para ejecutar el programa y asi ver las tablas creadas

In [ ]:
-- para poder ver las tablas creadas dentro de hive
show databases; --esto ya es codigo parecido a SQL, porque estas ejecutando codigo HQL dentro de Hive.
use integrador; --usamos la base de datos integrador
show tables; --para que nos muestre las tablas creadas
select * from calendario LIMIT 5; --para ver como esta esta tabla
select * from canal_venta LIMIT 5; --idem anterior
select * from cliente LIMIT 5; --idem anterior
select * from compra LIMIT 5; --idem anterior
select * from empleado LIMIT 5; --idem anterior
select * from gasto LIMIT 5; --idem anterior
select * from producto LIMIT 5; --idem anterior
select * from proveedor LIMIT 5; --idem anterior
select * from sucursal LIMIT 5; --idem anterior
select * from tipo_gasto LIMIT 5; --idem anterior
select * from venta LIMIT 5; --idem anterior

ATENCION: aca puede aparecer el primer error del integrador... si al final de la ejecucion dice Failed with exception java.io [...] /idtipodegasto=1/000000_0 returned false

Esto significa que ya intentaste hacer este paso y estas teniendo un problema por nomenclatura duplicada:

In [ ]:
exit
sudo docker exec -it namenode bash #ingresas al administrador de archivos
hdfs dfs -ls / #verificas si tenes la carpeta data2 creada
hdfs dfs -ls /data2/gasto/ #verificas si ya tenes las carpetas idtipogasto=1 creda y sucesivas
# de ser asi, seguis con lo siguiente
hdfs dfs -rm -r /data2/gasto/* #eliminando todas esas carpetas, -rm es para remover y -r es para indicar que sea recursivo hasta que elimine todos los archivos dentro de la carpeta y finalmente las carpetas
exit #para volver a la VM

#solucionado el error volvemos a intentarlo desde el inicio del ejercicio salteando el cp (copy)

ATENCION: tenemos un segundo error... que dice en este caso FAILED: SemanticException [Error 10044]: [...]

Esto es debido a que la tabla del datasets no se corresponde a la tabla creada en el script (diferente cantidad de columnas), lo corregimos y lo corremos manualmente (y terminamos de correr el script manual por facilidad, mas adelante veremos como modificar archivos):

In [ ]:
INSERT INTO calendario
SELECT
id,
fecha,
anio,
mes,
dia,
trimestre,
semana,
dia_nombre,
mes_nombre
FROM integrador.calendario;

DROP TABLE IF EXISTS proveedor;
CREATE EXTERNAL TABLE IF NOT EXISTS proveedor (
IDProveedor INTEGER,
Nombre VARCHAR(40),
Address VARCHAR(150),
City VARCHAR(80),
State VARCHAR(50),
Country VARCHAR(20),
departamen VARCHAR(50)
)
STORED AS PARQUET
LOCATION '/data2/proveedor'
TBLPROPERTIES ('parquet.compression'='SNAPPY');

INSERT INTO proveedor
SELECT
IdProveedor,
Nombre,
Direccion,
Ciudad,
Provincia,
Pais,
Departamento
FROM integrador.proveedor;

DROP TABLE trips_part;

-- No seguimos con las dos sentencias finales ya que no existe la base de datos asociada para cargar las particiones.

### Ejercicio 4: SQL e Indices

Si cerraste la sesion y te paso lo mismo que a mi, que las bases de dato en hive creadas en el punto anterior ya no existen, no te preocupes... vamos a aprovechar la situacion para aprender una herramienta adicional WinSCP.

Primero, vamos a realizar los cambios que vimos en el punto anterior sobre el archivo Paso03.hql dentro de nuestra PC Windows (si no lo descargaron, haganlo), esto quiere decir modificar la query de la linea 277 para que coincida con las columnas de la query de la linea 262; y elimnar las ultimas dos querys que no corrimos.

Una vez guardado el archivo, ejecutan el programa de Windows WinSCP, en cuyo caso les pedira que ingresen el hostname usual, asi como usuario y contrasena de la VM en cuestion. Finalmente aparecen en una pantalla partida, en donde en el lado izquierdo tienen el explorador de archivos de Windows y a la derecha, el de la VM. Luego, eliminan el Paso03.hql de la VM y copian el nuevo existente en Windows.

A partir de aqui, volvemos a correr los pasos 2 y 3 sobre la VM y no deberia haber ningun inconveniente, habiendo ademas aprendido a conectar ambos entornos de una manera sensilla para el intercambio de archivos.

A parir de ello, seguimos con el paso 4:

In [ ]:
sudo docker cp Paso04.hql hive-server:/opt/Paso04.hql
sudo docker exec -it hive-server bash
hive -f Paso04.hql

### Ejercicio 5: No-SQL

Este ejercicio es el mas extenso del practico, y conlleva aprender a usar las distintas aplicaciones para manejar bases de datos no relacionales.

#### HBase:

In [ ]:
sudo docker-compose -f docker-compose-v3.yml up -d #inicias cluster con hbase y otros.
sudo docker exec -it namenode bash #nos metemos en el namenode para enviar el archivo a leer por hbase a hdfs
cd home #nos metemos en la carpeta donde tenemos los Datasets
cd Datasets #nos metemos en la carpeta donde esta personal.csv
hdfs dfs -ls / #verificamos tener la carpeta hbase creada
hdfs dfs -mkdir /hbase #la creamos en caso de no tenerla
hdfs dfs -mkdir /hbase/data #idem para la otra carpeta dentro, se pueden crear juntar tambien pero para practicar nunca viene mal
hdfs dfs -put personal.csv /hbase/data/personal.csv #mandamos el archivo a hdfs
exit #para volver a la VM
sudo docker exec -it hbase-master hbase shell #para ingresar directamente a la aplicacion hbase dentro del contenedor de hbase-master
#hbase
create 'personal','personal_data'
list 'personal'
put 'personal',1,'personal_data:name','Juan'
put 'personal',1,'personal_data:city','Córdoba'
put 'personal',1,'personal_data:age','25'
put 'personal',2,'personal_data:name','Franco'
put 'personal',2,'personal_data:city','Lima'
put 'personal',2,'personal_data:age','32'
put 'personal',3,'personal_data:name','Ivan'
put 'personal',3,'personal_data:age','34'
put 'personal',4,'personal_data:name','Eliecer'
put 'personal',4,'personal_data:city','Caracas'
get 'personal','4'
exit
sudo docker exec -it hbase-master bash #para ingresar al contenedor hbase-master sin ingresar a la aplicacion hbase

hbase org.apache.hadoop.hbase.mapreduce.ImportTsv -Dimporttsv.separator=',' -Dimporttsv.columns=HBASE_ROW_KEY,personal_data:name,personal_data:city,personal_data:age personal hdfs://namenode:9000/hbase/data/personal.csv #con esto importas el archivo personal a la base de datos ya creada
hbase shell #te metes nuevamente en hbase aplicacion
scan 'personal' #escaneas la abse
create 'album','label','image' #creas otra base de datos de albumes, etiquetas e imagenes
put 'album','label1','label:size','10'
put 'album','label1','label:color','255:255:255'
put 'album','label1','label:text','Family album'
put 'album','label1','image:name','holiday'
put 'album','label1','image:source','/tmp/pic1.jpg'
get 'album','label1'
exit #salis de la aplicacion hbase 
exit #salis del contenedor hbase-master

#### MongoDB

En este punto aprenderemos a manejar MongoDB, cuya funcionalidad es muy similar a SQL pero el criterio de 'apilacion' de datos es por fila y no por columna, dejando la posibilidad de tener columnas variables (ya que el valor se relaciona con la fila y no con dicha columna que puede o no existir en otros registros).

In [ ]:
sudo docker-compose -f docker-compose-v3.yml up -d
sudo docker ps #verifican si el contenedor mongodb esta dado de alta, caso contrario pueden tener un tema con la ultima actualizacion de mongo
# SI NO LES ANDA MONGO: vamos a proceder a modificar el archivo yml para que copie una imagen de mongo mas vieja que podamos correr
vim docker-compose-v3.yml # con esto abrimos el archivo con el editor de texto vim de ubuntu
# bajamos con la flecha hasta donde dice mongoddb:
# luego, con cuidado, apretamos la tecla 'i' que abre el modo editor
# donde dice mongo:latest borramos latest y ponemos 4.0.4
# luego salimos del modo editor apretando 'esc'
# finalmente guardamos los cambios y salimos apretando las teclas ':wq' (: para opciones, w para sobreescribir el archivo y q para salir)
# LISTO, volvemos con los pasos normales
sudo docker-compose -f docker-compose-v3.yml up -d # para volver a ejecutar los contenedores y dar de alta mongodb
sudo docker ps #verificamos este mongodb
cd Datasets
sudo docker cp iris.csv mongodb:/data/iris.csv
sudo docker cp iris.json mongodb:/data/iris.json
sudo docker exec -it mongodb bash
mongoimport /data/iris.csv --type csv --headerline -d dataprueba -c iris_csv #para importar los archivos  a la base de datos dataprueba
mongoimport --db dataprueba --collection iris_json --file /data/iris.json --jsonArray
mongo #para ejecutar la aplicacion
use dataprueba
show collections
db.iris_csv.find()
db.iris_json.find()
exit #para salir de consola
mongoexport --db dataprueba --collection iris_csv --fields sepal_length,sepal_width,petal_length,petal_width,species --type=csv --out /data/iris_export.csv
mongoexport --db dataprueba --collection iris_json --fields sepal_length,sepal_width,petal_length,petal_width,species --type=json --out /data/iris_export.json
cd data
ls #para ver que se hayan exportado correctamente
exit
cd .. #para volver a la carpeta principal
cd Mongo
sudo docker cp mongo-hadoop-hive-2.0.2.jar namenode:/home/mongo-hadoop-hive-2.0.2.jar #copiando archivos de configuracion de imagen a namenode
sudo docker cp mongo-hadoop-core-2.0.2.jar namenode:/home/mongo-hadoop-core-2.0.2.jar
sudo docker cp mongo-hadoop-spark-2.0.2.jar namenode:/home/mongo-hadoop-spark-2.0.2.jar
sudo docker cp mongo-java-driver-3.12.11.jar namenode:/home/mongo-java-driver-3.12.11.jar
sudo docker exec -it namenode bash
cd home
hdfs dfs -put mongo-hadoop-hive-2.0.2.jar /tmp/udfs/mongo-hadoop-hive-2.0.2.jar #enviando los archivos al entorno hdfs donde todos los contenedores puedan acceder a ellos
hdfs dfs -put mongo-hadoop-core-2.0.2.jar /tmp/udfs/mongo-hadoop-core-2.0.2.jar
hdfs dfs -put mongo-hadoop-spark-2.0.2.jar /tmp/udfs/mongo-hadoop-spark-2.0.2.jar
hdfs dfs -put mongo-java-driver-3.12.11.jar /tmp/udfs/mongo-java-driver-3.12.11.jar
exit
sudo docker exec -it hive-server bash
hive #nos metemos en la aplicacion de hive
ADD JAR hdfs://namenode:9000/tmp/udfs/mongo-hadoop-hive-2.0.2.jar; #agrego los archivos de configuracion a hive
ADD JAR hdfs://namenode:9000/tmp/udfs/mongo-hadoop-core-2.0.2.jar;
ADD JAR hdfs://namenode:9000/tmp/udfs/mongo-hadoop-spark-2.0.2.jar;
ADD JAR hdfs://namenode:9000/tmp/udfs/mongo-java-driver-3.12.11.jar;
CREATE EXTERNAL TABLE iris
ROW FORMAT SERDE 'com.mongodb.hadoop.hive.BSONSerDe'
WITH SERDEPROPERTIES (
    'mongo.columns.mapping'='{
        "id":"_id","fila":"fila",
        "sepal_length":"sepal_length",
        "sepal_width":"sepal_width",
        "petal_length":"petal_length",
        "petal_width":"petal_width",
        "species":"species"}') 
TBLPROPERTIES (
    'mongo.uri'='mongodb://user:pass@<IP_Anfitrion>:27017/dataprueba.iris_csv');
#este codigo a mi me presenta error y creo que es porque el jar usa otra verion de mongodb que no puedo ejecutar o mismo otra de hive... no lo pude resolver, abierto a comentario y ayuda para resolucion!
exit;
exit 
cd .. #para volver a donde siempre

#### Neo4J

Es un programa que maneja No-SQL db mediante un conjunto de nodos.

Unicamente vamos a resolver el ejericio presente en ejemploNeo4J.txt porque es el mas complejo.

In [ ]:
cd Datasets
cd producto
sudo docker cp Producto.csv neo4j:/var/lib/neo4j/import/producto.csv
cd ..
cd cliente
sudo docker cp Cliente.csv neo4j:/var/lib/neo4j/import/cliente.csv
cd ..
cd venta
sudo docker cp Venta.csv neo4j:/var/lib/neo4j/import/venta.csv
cd ..
cd ..
sudo docker exec -it neo4j bash
cypher-shell #consola cypher de neo4j
neo4j # es el nombre de usuario
zeppelin #es la contrasena

In [ ]:
-- salteamos tipo de producto y todo lo relacionado porque la base de datos no existe en Datasets
LOAD CSV WITH HEADERS FROM "file:///producto.csv" AS row
CREATE (n:Producto)
SET n = row,
n.IdProducto = toInteger(row.IdProducto),
n.Producto = toString(row.Producto), 
n.IdTipoProducto = toInteger(row.IdTipoProducto);

LOAD CSV WITH HEADERS FROM "file:///cliente.csv" AS row
CREATE (n:Cliente)
SET n = row,
n.IdCliente = toInteger(row.IdCliente),
n.Nombre_y_Apellido = toString(row.Nombre_y_Apellido),
n.Domicilio = toString(row.Domicilio);

LOAD CSV WITH HEADERS FROM "file:///venta.csv" AS row
CREATE (n:Venta)
SET n = row,
n.IdVenta = toInteger(row.IdVenta),
n.IdCliente = toInteger(row.IdCliente),
n.IdProducto = toInteger(row.IdProducto),
n.Precio = toFloat(row.Precio),
n.Cantidad = toInteger(row.Cantidad);

MATCH(pro:Producto) RETURN pro LIMIT 5;
MATCH(cli:Cliente) RETURN cli LIMIT 5;
MATCH(vta:Venta) RETURN vta LIMIT 5; -- luego de correr estos tres, vemos que producto y cliente estan mal cargados (por la separacion de ; y no ,)
-- corrijamoslo!!!
MATCH (p:Producto) DETACH DELETE p;
MATCH (c:Cliente) DETACH DELETE c;

LOAD CSV WITH HEADERS FROM "file:///producto.csv" AS row FIELDTERMINATOR ';'
CREATE (n:Producto)
SET n = row,
n.IdProducto = toInteger(row.IdProducto),
n.Producto = toString(row.Producto), 
n.IdTipoProducto = toInteger(row.IdTipoProducto);

LOAD CSV WITH HEADERS FROM "file:///cliente.csv" AS row FIELDTERMINATOR ';'
CREATE (n:Cliente)
SET n = row,
n.IdCliente = toInteger(row.IdCliente),
n.Nombre_y_Apellido = toString(row.Nombre_y_Apellido),
n.Domicilio = toString(row.Domicilio);

CREATE INDEX FOR (p:Producto) ON (p.IdProducto);
CREATE INDEX FOR (c:Cliente) ON (c.IdCliente);
CREATE INDEX FOR (v:Venta) ON (v.IdVenta);

MATCH (p:Producto),(v:Venta)
WHERE p.IdProducto = v.IdProducto
CREATE (p)-[:VENDIDO_EN]->(v);

MATCH (c:Cliente),(v:Venta)
WHERE c.IdCliente = v.IdCliente
CREATE (c)-[:CLIENTE_EN]->(v);

MATCH (p:Producto)-[:VENDIDO_EN]->(v:Venta) RETURN p, v LIMIT 5;
MATCH (c:Cliente)-[:CLIENTE_EN]->(v:Venta) RETURN c, v LIMIT 5;
-- vemos que no se creo ninguna relacion, si inspeccionamos bien las consultas anteriores encontramos que el problema es que el campo IDproducto en una tabla es INT y en otra STR
MATCH (p:Producto) DETACH DELETE p;
MATCH (c:Cliente) DETACH DELETE c;

LOAD CSV WITH HEADERS FROM "file:///producto.csv" AS row FIELDTERMINATOR ';'
CREATE (n:Producto)
SET n = row,
n.ID_PRODUCTO = toInteger(row.ID_PRODUCTO),
n.Producto = toString(row.Producto), 
n.IdTipoProducto = toInteger(row.IdTipoProducto);

LOAD CSV WITH HEADERS FROM "file:///cliente.csv" AS row FIELDTERMINATOR ';'
CREATE (n:Cliente)
SET n = row,
n.ID = toInteger(row.ID),
n.Nombre_y_Apellido = toString(row.Nombre_y_Apellido),
n.Domicilio = toString(row.Domicilio);

MATCH (p:Producto),(v:Venta)
WHERE p.ID_PRODUCTO = v.IdProducto
CREATE (p)-[:VENDIDO_EN]->(v);

MATCH (c:Cliente),(v:Venta)
WHERE c.ID = v.IdCliente
CREATE (c)-[:CLIENTE_EN]->(v);

MATCH (p:Producto)-[:VENDIDO_EN]->(v:Venta) RETURN p, v LIMIT 5;
MATCH (c:Cliente)-[:CLIENTE_EN]->(v:Venta) RETURN c, v LIMIT 5;
:exit -- para salir y finalizar el ejercicio

#### Zeppelin

Zeppelin es una plataforma para colaborar conjuntamente conectados a un servidor que la ventaja que tiene es que puede correr multiples idiomas en un mismo script, solo le indicas el idioma que lleva la siguiente parte del texto mediante %"idioma" (sea file, neo4j, python, etc)... <br>
<ul>
<li>
Para conectarnos a zeppelin vamos a tener que averiguar algunas cosas del archivo docker compose y poner en el navegador web lo siguiente: http://"IP anfitrion":8888/ (el 8888 es el puerto de conexion que figura en el docker compose). <br> </li>
<li>
Dentro de dicha pagina ingresas en donde dice anonymus (arriba a la derecha) y apretas la opcion "interpreter". <br> </li>
<li>
Buscas el interpreter "file" y pones http://"IP_Anfitrion":9870/webhdfs/v1/ en la opcion hdfs.url y guardas. <br> </li>
<li>
Siguiendo eso te metes en el interprete neo4j y pones http://"IP_Anfitrion":7687 en neo4j.url, neo4j en neo4j.auth.user y zeppelin en neo4j.auth.password. <br> </li> </ul> 
Cumplidos dichos pasos, ya estas listo y con las conexiones activas para ejecutar codigo dentro de tu maquina virtual... podes correr el codigo de sudo docker que quieras y el de neo4j lo mismo y se ejecutan en la maquina virtual.
<br>

### Spark

#### Spark y Scala
En este caso vamos a usar Spark para ejecutar inicialmente Python y Scala... no hace falta hacer otra cosa que seguir los pasos en rajatabla para ejecutar sin inconvenientes.
Adicionalmente, se puede ver las aplicaciones en ejecucion y en los siguientes sitios: 
Spark master: http://<IP_Anfitrion>:8080/
Spark worker: http://<IP_Anfitrion>:8081/

#### Kafka


In [ ]:
sudo docker-compose up -d
sudo docker exec -it kafka_container bash
cd /opt/kafka/bin
sh kafka-topics.sh --create --bootstrap-server kafka:9092 --replication-factor 1 --partitions 100 --topic demo
sh kafka-topics.sh --list --bootstrap-server kafka:9092
sh kafka-topics.sh --describe --bootstrap-server kafka:9092 --topic demo 
sh kafka-console-consumer.sh --bootstrap-server kafka:9092 --topic demo --from-beginning
#Luego, hay que abrir otro putty y loguearnos con las mismas credenciales, en este otro putty hacer lo siguiente
sudo docker exec -it kafka_container bash
sh kafka-console-producer.sh --broker-list localhost:9092 --topic demo
#esto va a generar una especie de chat de un lado, en donde podras escribir en el putty abierto al final y el mensaje aparecera en el primer putty
<ctrl + c> # en ambos putty y hacemos exit en ambos, luego exit de nuevo en uno para cerrarlo y nos quedamos con uno dentro de spark-master

#### Comparativa Dataset y Dataframe en Scala

In [ ]:

spark/bin/spark-shell --master spark://spark-master:7077
#pegamos todo el script existente en el archivo puebaScala.scala salteando los puntos realcionados a airports y flightbyorigin ya que no tenemos las db
#luego salimos de scala con :quit e ingresamos a python
/spark/bin/pyspark --master spark://spark-master:7077
#seguimos ejecutando el codigo existente en los archivos Paso05.py y pruebaPySpark
#Luego debemos volver a copiar los datasets a namenode y enviarlos a hdfs nuevamente uno por uno porque los mismos fueron modificados
#a partir de alli se puede continuar ejecutando los archivos Paso06_GeneracionVentasNuevasPorDia.py y Paso06_IncrementalVentas.py
# a los fines practicos seguiremos sin este paso para evitar extensiones de pasos que ya vimos
#lo unico que deberian verificar es los nombres de las carpetas utilizadas en los archivos .py, y si quieren pueden cambiarlos mediante el editor de textos vim

### Ejercicio 8: Herramientas de orquestación de flujos de datos


In [ ]:
cd DS-M4-Herramientas_Big_data
sudo docker-compose down
cd ..
git clone https://github.com/sercasti/datalaketools.git #dentro de la maquina virtual
cd datalaketools
sudo docker-compose up -d
# en el navegador, poner http://<IP_Anfitrion>:5050/

Para hacerlo sensillo, no voy a hacer el ejericio 8 ya que su resolucion es perfecta sin que quepa ninguna duda en como proceder.

Espero hayan valorado este trabajo y pueda ayudarlos en completar su proyecto integrador.